In [1]:
from gpx_converter import Converter
from mapbox import MapMatcher
import numpy as np
from geopy.distance import geodesic

In [2]:
#gpxfile="./01-runkeeper-data-export-2021-05-03-034233/2021-05-03-110913.gpx"
gpxfile="./01-runkeeper-data-export-2021-05-04-084136/2021-04-18-122351.gpx"
dic = Converter(input_file=gpxfile).gpx_to_dictionary(latitude_key='latitude', longitude_key='longitude')

In [3]:
latlon=[]
for i,j in zip(dic["latitude"],dic["longitude"]):
    latlon.append([j,i])
    
#datetimeはjson型にできないためstrに変換
dic["time"]=[i.isoformat() for i in dic["time"]]

In [4]:
service = MapMatcher(access_token="XXXX")

In [5]:
#response = service.match(line, profile='mapbox.driving')
#response = service.match(line, profile='mapbox.walking')

In [6]:
#map matching api は一度に100件までしか処理できないため分割処理
corrected=[]
dist=0
for i in range(len(latlon)//100+1):
    line={}
    line["type"]="Feature"
    line["properties"]={"coordTimes":dic["time"][i*100:(i+1)*100]}
    line["geometry"]={"type": "LineString","coordinates":latlon[i*100:(i+1)*100]}
    
    response = service.match(line, profile='mapbox.walking')
    print(i,response)
    
    #結果を統合
    for i in response.geojson()['features']:
        corrected.extend(i["properties"]["matchedPoints"])
        dist+=i["properties"]["distance"]

0 <Response [200]>
1 <Response [200]>
2 <Response [200]>
3 <Response [200]>
4 <Response [200]>
5 <Response [200]>
6 <Response [200]>
7 <Response [200]>
8 <Response [200]>
9 <Response [200]>


In [7]:
#foliumに入力するために緯度経度を逆に
np_corrected=np.array(corrected)
re_corrected=np.zeros_like(np_corrected)
re_corrected[:,1]=np_corrected[:,0]
re_corrected[:,0]=np_corrected[:,1]

In [8]:
#元データ表示用
co2=[]
for lon,lat in zip(dic["longitude"],dic["latitude"]):
    co2.append([lat,lon])

In [9]:
import folium
map_osm = folium.Map(location=[35.6,139.7],zoom_start=11)

folium.PolyLine(
        locations=[re_corrected],
        color="red"
        ).add_to(map_osm)

folium.PolyLine(
        locations=co2,
        color="blue"
        ).add_to(map_osm)
    
map_osm

In [10]:
#距離測定
dist_rawbase=0
for num in range(len(co2)-1):
    dist_rawbase+= geodesic(co2[num], co2[num+1]).m

dist_correctbase=0
for num in range(len(re_corrected)-1):
    dist_correctbase+= geodesic(re_corrected[num], re_corrected[num+1]).m

In [11]:
dist_rawbase

9937.8056134017

In [12]:
dist_correctbase

10037.057959072488